In [74]:
import pandas as pd
import numpy as np
import plotly.express as px
from pathlib import Path
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### IMPORT DATA

In [75]:
RAW_DATASET_PATH = "data/postings.csv"
raw_data = pd.read_csv(RAW_DATASET_PATH)

In [76]:
raw_data.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')

In [77]:
from ydata_profiling import ProfileReport

In [78]:
PROFILE_REPORT_PATH="eda_job_postings_dataset.html"

def generate_profile_report(df: pd.DataFrame, path: Path = PROFILE_REPORT_PATH) -> None:
    profile = ProfileReport(df, title="LinkedIn Job Postings")
    profile.to_notebook_iframe()
    profile.to_file(PROFILE_REPORT_PATH)

# generate_profile_report(raw_data)

### UTILITIES

In [79]:
def save_file(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)

def load_file(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

### DATA PREPROCESSING

In [80]:
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import re
import json
import pickle
import os

##### Keep only a subset of the columns for our needs

In [81]:
COLUMNS_TO_KEEP = ['company_name'
                   ,'title'
                   ,'description'
                   ,'pay_period'
                   ,'max_salary'
                   ,'med_salary'
                   ,'min_salary'
                   ,'location'
                   ,'remote_allowed'
                   ,'work_type'
                   ,'currency']
jobs_data = raw_data[COLUMNS_TO_KEEP]

##### Create a new yearly salary column from existing salary columns to standardize this metric across the dataset

In [82]:
salary_columns: list[str] = ["max_salary","med_salary","min_salary"]
salary_period_type_column: str = "pay_period"

def convert_to_yearly_salary(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df.loc[df[salary_period_type_column]=="HOURLY", salary_columns] = df.loc[df[salary_period_type_column]=="HOURLY", salary_columns]*2080
    df.loc[df[salary_period_type_column]=="WEEKLY", salary_columns] = df.loc[df[salary_period_type_column]=="WEEKLY", salary_columns]*52
    df.loc[df[salary_period_type_column]=="BIWEEKLY", salary_columns] = df.loc[df[salary_period_type_column]=="BIWEEKLY", salary_columns]*26
    df.loc[df[salary_period_type_column]=="MONTHLY", salary_columns] = df.loc[df[salary_period_type_column]=="MONTHLY", salary_columns]*12

    df["standardized_salary"] = df["med_salary"]
    
    df["avg_min_max"] = (df["max_salary"]+df["min_salary"])/2
    df.loc[df["standardized_salary"].isna()==True, "standardized_salary"] = df.loc[df["standardized_salary"].isna()==True,"avg_min_max"]

    return df

In [83]:
jobs_data = convert_to_yearly_salary(jobs_data)
jobs_data = jobs_data[jobs_data["standardized_salary"]>=0]
jobs_data = jobs_data.drop(columns=["max_salary","med_salary","min_salary","pay_period","avg_min_max"])

##### Filter data on usd currency

In [84]:
# value_counts: 
# USD    36058
# EUR        6
# CAD        3
# BBD        2
# AUD        2
# GBP        2

jobs_data = jobs_data[jobs_data["currency"]=="USD"]
jobs_data = jobs_data.drop(columns="currency")

##### Focus on full_time, contract, part_time work types

In [85]:
# value_counts: 
# FULL_TIME     29119
# CONTRACT       3848
# PART_TIME      2304
# TEMPORARY       394
# INTERNSHIP      247
# OTHER           138
# VOLUNTEER         8

jobs_data = jobs_data[jobs_data["work_type"].isin(["FULL_TIME","CONTRACT","PART_TIME"])]

##### Plot standardized salaries

In [86]:
px.box(jobs_data,x="standardized_salary",y="work_type")

In [87]:
jobs_data = jobs_data[(jobs_data["standardized_salary"]>jobs_data["standardized_salary"].quantile(0.05))&(jobs_data["standardized_salary"]<jobs_data["standardized_salary"].quantile(0.99))]

In [88]:
px.histogram(jobs_data["standardized_salary"])

##### Concatenate information-rich text columns

In [89]:
COLUMNS_TO_CONCATENATE = ['company_name', 'title', 'description']

jobs_data["title"] = jobs_data["title"].str.strip()
jobs_data[COLUMNS_TO_CONCATENATE] = jobs_data[COLUMNS_TO_CONCATENATE].fillna("-",)
jobs_data["augmented_description"] =  jobs_data[COLUMNS_TO_CONCATENATE].agg(' '.join, axis=1)

jobs_data = jobs_data.drop(columns=["company_name","title","description"])

##### Convert 'remote_allowed' column to a boolean column

In [90]:
jobs_data["remote_allowed"] = jobs_data["remote_allowed"].fillna(0).astype(int)

In [91]:
jobs_data = pd.get_dummies(jobs_data,columns=["work_type"])

##### Location column is the following shape: city,state. In order to avoid to much values in the location columns, we only keep the state column. However, the state column contains too many values too, so we replace each value by its frequency.

In [92]:
tmp_location = jobs_data["location"].str.split(',',expand=True) 
tmp_location.loc[tmp_location[1].isna(),1] = tmp_location.loc[tmp_location[1].isna(),0] 
jobs_data["state"] = tmp_location[1].str.strip()

with open('location_renaming_mapping.json', 'r') as json_file:
    location_renaming_mapping = json.load(json_file)

jobs_data["state"] = jobs_data["state"].replace(location_renaming_mapping)

jobs_data = jobs_data.drop(columns=["location"])

##### Remove stopwords in augmented description

In [93]:
stop_words_path = Path("stop_words.pkl")

if not os.path.exists(stop_words_path):
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    save_file(stop_words,stop_words_path)
else:
    stop_words = load_file(stop_words_path)

In [94]:
px.histogram(jobs_data['augmented_description'].apply(lambda x: len(x.split(' '))))

In [95]:
def clean_description(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

jobs_data['augmented_description'] = jobs_data['augmented_description'].apply(clean_description)

In [96]:
px.histogram(jobs_data['augmented_description'].apply(lambda x: len(x.split(' '))))

### Embed augmented description

In [97]:
def join_embeddings_to_df(jobs_data,augmented_description_embeddings):
    jobs_data = jobs_data.reset_index(drop=True)
    columns_embeddings_df = [str(c) for c in range(augmented_description_embeddings.shape[1])]
    return pd.concat([jobs_data,pd.DataFrame(augmented_description_embeddings, columns=columns_embeddings_df)],axis=1,join='inner')

In [98]:
model = SentenceTransformer("all-MiniLM-L6-v2")
augmented_description_embeddings_path = Path("augmented_description_embeddings.pkl")


if not os.path.exists(augmented_description_embeddings_path):
    augmented_description_list = jobs_data["augmented_description"].to_list()
    augmented_description_embeddings = model.encode(augmented_description_list)
    jobs_data = jobs_data.drop(columns=["augmented_description"])
    save_file(augmented_description_embeddings,"augmented_description_embeddings.pkl")
    jobs_data = join_embeddings_to_df(jobs_data,augmented_description_embeddings)

else:
    jobs_data = jobs_data.drop(columns=["augmented_description"])
    augmented_description_embeddings = load_file(augmented_description_embeddings_path)
    jobs_data = join_embeddings_to_df(jobs_data,augmented_description_embeddings)


In [99]:
preprocessed_data_path = Path("data/preprocessed_postings.csv")

if not os.path.exists(preprocessed_data_path):
    jobs_data.to_csv(path_or_buf=preprocessed_data_path,index=False)

### Pipeline

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA

from scipy.stats import randint, uniform, loguniform

import mlflow
from mlflow.models import infer_signature

from tqdm import tqdm
import dill

os.environ["TOKENIZERS_PARALLELISM"] = "false"

##### Train/Test split

In [101]:
X = jobs_data.drop(columns=["standardized_salary"])
y = jobs_data["standardized_salary"]

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=42)

##### Transformers

In [102]:
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.freq_map = {}

    def fit(self, X: pd.Series, y=None):
        # Compute state frequency on training set
        self.freq_map = X.value_counts(normalize=False).to_dict()
        return self

    def transform(self, X: pd.Series):
        # Encode the states in the test data set according to the frequencies calculated in the training set.
        # If a state was not in the training set, we assign it a frequency of 1.
        freq_map_ = self.freq_map.copy()
        
        for x in X: 
            if x not in self.freq_map.keys():
                freq_map_[x] = 1

        return pd.DataFrame(X.map(freq_map_))
    
    def set_output(self, transform="pandas"):
        self._transform_output = transform
        return self
    
def frequency_transformer() -> ColumnTransformer:
    return ColumnTransformer(transformers=[('frequency_encoding',FrequencyEncoder(),'state')]
                             ,remainder='passthrough'
                             ,verbose_feature_names_out=False)

##### Estimators

In [103]:
# Tracking URI of the Mlflow server.
MLFLOW_TRACKING_URI = "http://127.0.0.1:8080"

# This command sets the tracking URI for the current session
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [104]:
def make_pipeline_with_model(model: BaseEstimator) -> Pipeline:
    pipeline = Pipeline(
        [
            ('frequency_tranformer',frequency_transformer()),
            ('pca',PCA()),
            ('model',model)
        ]
    )
    pipeline.set_output(transform="pandas")
    return pipeline

##### Estimators -> Baseline

In [105]:
def model_hyp_optimisation(
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    model: BaseEstimator,
    param: dict,
    k_fold:int,
    n_iter:int,
    experiment_name: str,
    run_name: str
) -> BaseEstimator:

    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=run_name):
        pipeline = make_pipeline_with_model(model)
        cv = RandomizedSearchCV(pipeline,param_distributions=param,n_iter=n_iter,cv=k_fold,scoring="neg_root_mean_squared_error",refit=True)
        best_model = cv.fit(X_train,y_train)

        mlflow.log_metric("rmse",-best_model.best_score_)
        mlflow.log_params(best_model.best_params_)

        mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path=model.__class__.__name__,
            signature=infer_signature(X_train, best_model.predict(X_train)),
            input_example=X_train,
            registered_model_name=model.__class__.__name__)
        
    return best_model

In [106]:
# model_hyp_optimisation(X_train=X_train
#                        ,y_train=y_train
#                        ,model=HistGradientBoostingRegressor()
#                        ,param={}
#                        ,k_fold=10
#                        ,n_iter=1
#                        ,experiment_name="Model selection"
#                        ,run_name="Baseline")

##### Estimators -> Model selection

##### Estimators -> Model selection -> Refining search space (Models & Hyperparameters)

In [107]:
def refine_ss_randomsearchcv(hyperparameter_grid,cv_results_path,n_iter=50,cv=10):

    cv_results: list[(str, pd.DataFrame)] = []

    for mp in tqdm(hyperparameter_grid, desc="Hyperparameter Tuning"):
        model, param = mp

        pipeline = make_pipeline_with_model(model)
        random_search = RandomizedSearchCV(pipeline, param, n_iter=n_iter, cv=cv, scoring="neg_root_mean_squared_error", n_jobs=-1)
        random_search.fit(X_train, y_train)

        cv_results.append((model.__class__.__name__, random_search.cv_results_))

    if not os.path.exists(cv_results_path):
        save_file(cv_results,cv_results_path)
    else:
        cv_results = load_file(cv_results_path)

    return cv_results

In [108]:
pd.set_option('display.max_colwidth', None)

def display_top_performers(cv_results_path):
    cv_results = load_file(cv_results_path)
    for i in range(len(cv_results)):
        print(f"MODEL : {cv_results[i][0]}")
        top_five_hypp_performers = pd.DataFrame(cv_results[i][1]).nlargest(n=5,columns="mean_test_score")
        display(pd.concat([top_five_hypp_performers.loc[:,"std_score_time":"params"].iloc[:,1:-1],top_five_hypp_performers[["mean_test_score","std_test_score"]]],axis=1))

##### 1st experience

In [109]:
cv_results_path = Path("cv_experiments/cv_results_v1.pkl")

if not os.path.exists(cv_results_path):
    hyperparameter_grid = [(MLPRegressor(), {
                                'pca__n_components': randint(10,150),
                                'model__hidden_layer_sizes': [(100,100),(100,150),(150,100),(150,150)],
                                'model__learning_rate_init': loguniform(1e-4, 1),
                                'model__learning_rate': ["constant", "adaptive"],
                                'model__alpha': loguniform(1e-4, 1)
                            }), (Ridge(), {
                                'pca__n_components': randint(10,150),
                                'model__alpha': uniform(loc=0,scale=3)
                            }), (KNeighborsRegressor(), {
                                'pca__n_components': randint(10,150),
                                'model__n_neighbors': randint(5,30), 
                            }), (DecisionTreeRegressor(), {
                                'pca__n_components': randint(10,150),
                                'model__max_depth': randint(5,50), 
                                'model__min_samples_split': randint(5,20), 
                                'model__min_samples_leaf': randint(5,50) 
                            }), (HistGradientBoostingRegressor(), {
                                'pca__n_components': randint(10,150),
                                'model__learning_rate': loguniform(1e-4, 1),
                                'model__max_iter': randint(5,50), 
                                'model__max_leaf_nodes': randint(5,200),
                                'model__min_samples_leaf': randint(5,50)
                            })]

    cv_results = refine_ss_randomsearchcv(hyperparameter_grid,cv_results_path,n_iter=50,cv=10)

In [110]:
display_top_performers(cv_results_path)

MODEL : MLPRegressor


,param_model__alpha,param_model__hidden_layer_sizes,param_model__learning_rate,param_model__learning_rate_init,param_pca__n_components,mean_test_score,std_test_score
20,0.736,"(100, 150)",adaptive,0.001,149,-37361.467,751.000
21,0.000,"(150, 150)",adaptive,0.001,97,-37879.762,717.779
37,0.017,"(100, 100)",constant,0.002,79,-37945.718,720.738
4,0.007,"(150, 150)",constant,0.001,94,-38016.055,818.594
8,0.006,"(150, 100)",constant,0.001,84,-38025.341,843.628


MODEL : Ridge


,param_model__alpha,param_pca__n_components,mean_test_score,std_test_score
27,2.396,147,-37447.828,694.626
18,0.508,147,-37449.156,687.117
23,1.634,143,-37460.644,691.304
4,2.087,143,-37461.119,693.027
16,2.685,142,-37469.033,693.038


MODEL : KNeighborsRegressor


,param_model__n_neighbors,param_pca__n_components,mean_test_score,std_test_score
24,7,127,-39163.890,726.547
30,11,132,-39196.195,763.710
23,9,129,-39207.775,723.090
12,11,104,-39227.298,726.356
5,11,80,-39240.179,766.119


MODEL : DecisionTreeRegressor


,param_model__max_depth,param_model__min_samples_leaf,param_model__min_samples_split,param_pca__n_components,mean_test_score,std_test_score
2,13,37,7,28,-42055.411,682.177
48,40,42,10,64,-42325.964,566.329
23,16,35,10,54,-42376.282,694.699
31,33,41,10,10,-42426.549,856.956
13,44,46,9,73,-42456.895,556.364


MODEL : HistGradientBoostingRegressor


,param_model__learning_rate,param_model__max_iter,param_model__max_leaf_nodes,param_model__min_samples_leaf,param_pca__n_components,mean_test_score,std_test_score
14,0.087,49,174,7,132,-35263.102,722.155
47,0.243,46,197,26,101,-35397.132,688.896
34,0.124,38,159,36,148,-35420.473,621.331
24,0.070,49,120,31,69,-36296.955,694.375
21,0.459,24,130,34,94,-37731.462,557.522


KEY LEARNINGS FOLLOWING THE 1st EXPERIENCE:

1. **We want to focus on the top three models:**   
    * (1) HistGradientBoostingRegressor       
    * (2) MLPRegressor   
    * (3) Ridge
2. **PCA:**    
    * n_components: (10,150) -> (100,250)
3. **MLPRegressor:**    
    * learning_rate_init: loguniform(1e-4, 1) -> loguniform(1e-4, 1e-2)
4. **Ridge:**    
    * alpha: uniform(loc=0,scale=3) -> uniform(loc=0.5,scale=3)
5. **HistGradientBoostingRegressor**
    * max_iter: randint(5,50) -> randint(50,150)    
    * max_leaf_nodes: randint(5,200) -> randint(20,250)

##### 2nd experience

In [111]:
cv_results_path = Path("cv_experiments/cv_results_v2.pkl")

if not os.path.exists(cv_results_path):
    hyperparameter_grid = [(MLPRegressor(), {
                                'pca__n_components': randint(100,250),
                                'model__hidden_layer_sizes': [(100,100),(100,150),(150,100),(150,150)],
                                'model__learning_rate_init': loguniform(1e-4, 1e-2),
                                'model__learning_rate': ["constant", "adaptive"],
                                'model__alpha': loguniform(1e-4, 1)
                            }), (Ridge(), {
                                'pca__n_components': randint(100,250),
                                'model__alpha': uniform(loc=0.5,scale=3)
                            }), (HistGradientBoostingRegressor(), {
                                'pca__n_components': randint(100,250),
                                'model__learning_rate': loguniform(1e-4, 1),
                                'model__max_iter': randint(50,150), 
                                'model__max_leaf_nodes': randint(50,250),
                                'model__min_samples_leaf': randint(5,50)
                            })]

    cv_results = refine_ss_randomsearchcv(hyperparameter_grid,cv_results_path,n_iter=100,cv=10)

In [112]:
display_top_performers(cv_results_path)

MODEL : MLPRegressor


,param_model__alpha,param_model__hidden_layer_sizes,param_model__learning_rate,param_model__learning_rate_init,param_pca__n_components,mean_test_score,std_test_score
96,0.008,"(100, 100)",adaptive,0.001,232,-36725.886,746.822
59,0.002,"(100, 100)",adaptive,0.002,230,-36760.266,736.324
61,0.411,"(100, 150)",constant,0.002,237,-36795.293,867.979
33,0.000,"(150, 150)",adaptive,0.001,215,-36871.552,680.664
37,0.000,"(150, 150)",constant,0.001,231,-36885.867,727.605


MODEL : Ridge


,param_model__alpha,param_pca__n_components,mean_test_score,std_test_score
74,3.155,249,-36681.092,669.950
97,1.292,248,-36697.710,663.582
30,1.199,245,-36702.958,663.360
95,2.172,244,-36714.004,664.400
92,2.439,242,-36758.066,667.333


MODEL : HistGradientBoostingRegressor


,param_model__learning_rate,param_model__max_iter,param_model__max_leaf_nodes,param_model__min_samples_leaf,param_pca__n_components,mean_test_score,std_test_score
61,0.082,112,203,46,151,-33820.291,667.911
10,0.141,149,195,34,168,-33831.518,535.854
36,0.096,111,195,23,100,-33904.803,722.030
55,0.054,124,206,23,165,-33930.149,727.363
11,0.120,138,91,5,206,-33993.376,610.094


KEY LEARNINGS FOLLOWING THE 2nd EXPERIENCE:

1. **We want to focus on the best models:**   
    * HistGradientBoostingRegressor     
2. **HistGradientBoostingRegressor**
    * learning_rate: loguniform(1e-4, 1) -> loguniform(1e-3, 1)
    * max_iter: randint(50,150) -> randint(110,200)    
    * max_leaf_nodes: randint(20,250) -> randint(90,250)
    * min_samples_leaf: randint(5,50) -> randint(20,70)

##### 3rd experience

In [113]:
cv_results_path = Path("cv_experiments/cv_results_v3.pkl")

if not os.path.exists(cv_results_path):
    hyperparameter_grid = [(HistGradientBoostingRegressor(), {
                                'pca__n_components': randint(100,250),
                                'model__learning_rate': loguniform(1e-3, 1),
                                'model__max_iter': randint(110,200), 
                                'model__max_leaf_nodes': randint(90,250),
                                'model__min_samples_leaf': randint(20,70)
                            })]

    cv_results = refine_ss_randomsearchcv(hyperparameter_grid,cv_results_path,n_iter=200,cv=10)

In [114]:
display_top_performers(cv_results_path)

MODEL : HistGradientBoostingRegressor


,param_model__learning_rate,param_model__max_iter,param_model__max_leaf_nodes,param_model__min_samples_leaf,param_pca__n_components,mean_test_score,std_test_score
148,0.076,172,202,67,151,-33388.694,737.270
131,0.050,190,243,65,205,-33446.684,677.483
174,0.083,188,118,26,238,-33449.204,794.053
199,0.055,177,190,31,223,-33487.775,759.575
108,0.087,179,201,38,130,-33488.814,634.125


KEY LEARNINGS FOLLOWING THE 3rd EXPERIENCE:
  
* **HistGradientBoostingRegressor**
    * learning_rate: loguniform(1e-3, 1) -> loguniform(1e-2, 1)
    * max_iter: randint(110,200) -> randint(170,220)    
    * max_leaf_nodes: randint(100,250) -> randint(190,250)
    * min_samples_leaf: randint(20,70) -> randint(30,70)

##### Estimators -> Model selection -> Hyperparameters optimisation

In [115]:
model_path = Path("models/best_model.pkl")

if not os.path.exists(model_path):
    model= HistGradientBoostingRegressor()
    hyperparameter_grid = {
                            'pca__n_components': randint(100,250),
                            'model__learning_rate': loguniform(1e-2, 1),
                            'model__max_iter': randint(170,220), 
                            'model__max_leaf_nodes': randint(90,250),
                            'model__min_samples_leaf': randint(20,70)
                        }

    best_model = model_hyp_optimisation(X_train
                                        ,y_train
                                        ,model
                                        ,hyperparameter_grid
                                        ,k_fold=10
                                        ,n_iter=10
                                        ,experiment_name="Hist Gradient Boosting Regressor"
                                        ,run_name="")

    with open(model_path, "wb") as f:
        dill.dump(best_model, f)